In [1]:
#importing the requests library
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import yfinance as yf #using the yahoo finance library 
import yahoo_fin.stock_info as si #using the yahoo_fin library 
import warnings
warnings.filterwarnings('ignore')

Getting stock data using the open source yahoo finance library. More information can be found [here](https://pypi.org/project/yfinance/)

In [34]:
#function to get stock data of a company


def stock_info(ticker):
    #define dates for 5 years
    start_date = '2017-01-01'
    end_date = '2022-12-31'
    df = yf.download(ticker,start=start_date,end=end_date)
    #plot the stock price over the years
    fig,ax=plt.subplots(figsize=(12,8))
    df['Adj Close'].plot(ax=ax);
    ax.set_title(f'Stock price of {ticker} from 2017 to 2022');
    plt.show();
   
    return fig,ax

In [33]:
#function to calculate stock return

def stock_return(ticker):
    start_date = '2017-01-01'
    end_date = '2022-12-31'
    df = yf.download(ticker,start=start_date,end=end_date)
    df_return = df[['Adj Close']]
    df_return['pct_change'] = df_return['Adj Close'].pct_change() # use pct change to calculate the diff 
    df_return.drop(df_return.index[0],inplace=True) # drop the Nan value from the first row
    
    #plot the returns over 5 years
    fig,ax = plt.subplots(figsize=(15,8))
    ax.plot(df_return['pct_change']);
    ax.set_title('Stock Return Pct from 2017 to 2022:' +' ' + ticker);
    plt.show()
    
    return fig,ax;
      

In [4]:
def calculate_beta(ticker):
    symbols = [ticker,'SPY']
    data = yf.download(symbols, start = '2019-01-01',end = '2022-12-31')['Adj Close']
    price_change = data.pct_change()
    price_change.drop(price_change.index[0],inplace=True)
    
    X = np.array(price_change[ticker]).reshape((-1,1))
    y = np.array(price_change['SPY'])

    X_train,X_test,y_train,y_test = train_test_split(X,y)
    lr = LinearRegression()
    lr.fit(X_train,y_train)

    y_preds = lr.predict(X_test)

    fig,ax = plt.subplots(figsize=(12,7));
    ax.plot(X_test,y_preds,linestyle=':',color='orange');
    ax.scatter(y_test,y_preds,alpha=0.5);
    ax.set_title(f' Beta value = {lr.coef_}');
    plt.show()
    
    return fig,ax

## stock evaluation

2. p/e ratio
3. p/b ratio
4. div yield

In [43]:
analysis = si.get_analysts_info('AAPL')
analysis

{'Earnings Estimate':   Earnings Estimate  Current Qtr. (Mar 2023)  Next Qtr. (Jun 2023)  \
 0   No. of Analysts                    27.00                 26.00   
 1     Avg. Estimate                     1.43                  1.24   
 2      Low Estimate                     1.37                  1.02   
 3     High Estimate                     1.56                  1.35   
 4      Year Ago EPS                     1.52                  1.20   
 
    Current Year (2023)  Next Year (2024)  
 0                38.00             36.00  
 1                 5.97              6.60  
 2                 5.38              5.62  
 3                 6.60              7.16  
 4                 6.11              5.97  ,
 'Revenue Estimate':           Revenue Estimate Current Qtr. (Mar 2023) Next Qtr. (Jun 2023)  \
 0          No. of Analysts                      24                   23   
 1            Avg. Estimate                  92.98B               85.55B   
 2             Low Estimate           

In [44]:
analysis['Earnings History']

,Earnings History,3/30/2022,6/29/2022,9/29/2022,12/30/2022
0,EPS Est.,1.43,1.16,1.27,1.94
1,EPS Actual,1.52,1.2,1.29,1.88
2,Difference,0.09,0.04,0.02,-0.06
3,Surprise %,6.30%,3.40%,1.60%,-3.10%
